### PART 02. 프롬프트와 출력 파서

#### Chapter5. 프롬프트

#### 1. 프롬프트 탬플릿 만들기

##### - 프롬프트 탬플릿의 구조
- 지시 사항(Instruction), 질문(Question), 문맥(Context)로 구성

##### - PromptTemplate 객체
- 프롬프트 템플릿 객체

- `template` : 프롬프트 문자열(지시사항, 질문, 문맥 등 지정)

- `input_variables` : 입력해야할 변수 목록(명), LangChain이 자동 추출(from_template()로)

- `partial_variables` : 사전 입력해둔 변수 목록(명)

##### - 프롬프트 템플릿 만드는 방법
방법 1. from_template() 메서드를 사용하여 PromptTemplate 객체 생성

- 치환될 변수를 `placeholder`로 생성하여 `{ }`로 묶어서 넣을 문자열 정의

In [ ]:
from dotenv import load_dotenv

load_dotenv()

# LangSmith 추적을 설정합니다. https://smith.langchain.com
#!pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH02-Prompt")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

`template` 매개변수에 값을 치환하여 생성(렌더링)하는 방법

- format 메서드 사용(수동 렌더링) : 사용자가 직접 PromptTemplate의 변수를 치환하여 문자열 프롬프트 생성

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?" # country는 단순 변수명

# from_template 메서드로 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# 단순 prompt 생성, format 메서드로 변수에 값을 치환
# 문자열 프롬프트 반환(PromptTemplate 객체 X)
prompt = prompt.format(country="대한민국")

- chain 생성 후 invoke 메서드 사용(자동 렌더링)
 : LangChain이 입력 값을 받아 PromptTemplate 내부에서 format 메서드 실행

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의. {country}는 변수로, 이후에 값이 들어갈 자리를 의미
template = "{country}의 수도는 어디인가요?" # country는 단순 변수명

# from_template 메서드로 PromptTemplate 객체 생성
prompt = PromptTemplate.from_template(template)

# chain 생성
chain = prompt | llm

# country 변수에 입력된 값이 자동으로 치환되어 수행됨
# ChatOpenAI는 메시지 객체(AIMessage)를 반환하여 content 속성으로 접근
chain.invoke("대한민국").content


방법 2. PromptTemplate 객체 생성과 동시에 prompt 생성
- `template`과 `input_variables` 매개변수 직접 지정

- `template`이 받는 인자가 1개인 경우

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{country}의 수도는 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country"], # input_variables값을 직접 지정, country 변수가 아닐 경우 error 발생
)

# prompt 생성
prompt.format(country="대한민국")

- `template`이 받는 인자가 여러개인 경우

In [ ]:
from langchain_core.prompts import PromptTemplate

# template 정의
template = "{country1}과 {country2}의 수도는 각각 어디인가요?"

# PromptTemplate 객체를 활용하여 prompt_template 생성
prompt = PromptTemplate(
    template=template,
    input_variables=["country1"],
    partial_variables={
        "country2": "미국"  # country2 값 미리 지정
    },
)

prompt.format(country1="대한민국")
prompt_partial = prompt.partial(country2="캐나다") # partial 메서드로 기존 PromptTemplate 객체를 복사하여 country2 값 변경
prompt_partial.format(country1="대한민국")

chain = prompt_partial | llm

chain.invoke("대한민국").content
chain.invoke({"country1": "대한민국", "country2": "호주"}).content

##### - 부분 변수 활용하기
: 사용자 정의 함수를 통해 프롬프트의 일부를 미리 설정 ex) 현재 날짜 자동 출력



In [ ]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate

# 오늘 날짜를 출력
datetime.now().strftime("%B %d")

# 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

prompt = PromptTemplate(
    template="오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해 주세요. 생년월일을 표기해주세요.",
    input_variables=["n"],
    partial_variables={
        "today": get_today  # today 함수 미리 지정
    },
)

# prompt 생성
prompt.format(n=3)

# chain 생성
chain = prompt | llm

print(chain.invoke(3).content)
print(chain.invoke({"today": "Jan 02", "n": 3}).content) # 각 매개변수 값을 직접 지정(today 함수 무시)
# ChatModel(ChatOpenAI) 사용 시 반환되는 AIMessage 객체에서 content로 텍스트를 추출

##### - YAML 파일로 프롬프트 템플릿 로드하기

YAML(YAML Ain’t Markup Language)
 : 

 → load_prompt()로 YAML 파일 불러옴 

In [ ]:
from langchain_core.prompts import load_prompt
from langchain_teddynote.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml", encoding="utf-8")
prompt.format(fruit="사과") # 프롬프트에 값 치환

prompt2 = load_prompt("prompts/capital.yaml")
print(prompt2.format(country="대한민국"))

#### 2. ChatPromptTemplate

##### - ChatPromptTemplate 정의
: 여러 메시지를 조합하여 대화형 프롬프트를 정의하는 템플릿 객체

→ 튜플 형식으로 `(role, message)`형태로 구성

→ MessagePromptTemplate를 모아 프롬프트를 설계하고,
실행 시 Message 객체로 변환하여 LLM에 전달

##### - Role의 구성요소
- `system` : 시스템 설정 메시지로 표현  

  → 모델의 말투, 금지사항, 역할, 출력 형식 등 대화 전체에 적용되는 전역 지시사항을 정의

  ex) 규칙, 페르소나, 금지사항 지정

  → 프롬프트 설계 시 `SystemMessagePromptTemplate`, 실행 시 `SystemMessage`객체로 전환

- `human` : 사용자 입력 메시지로 표현  

  → 실제 사용자 발화 또는 대화 맥락을 구성하기 위한 사용자 메시지

  → 프롬프트 설계 시 `HumanMessagePromptTemplate`, 실행 시 `HumanMessage`객체로 전환

- `ai` : AI 응답 메시지 (`AIMessage` 객체)로 표현  

  → 모델이 어떤 방식으로 응답해야 하는지를 보여주는 예시 메시지로 활용될 수 있으며, 응답 스타일·형식·톤을 유도하는 역할을 수행
  
  → 퓨샷(few-shot)학습 효과를 유도함

  → 프롬프트 설계 시 `AIMessagePromptTemplate`, 실행 시 `AIMessage`객체로 전환

##### - PromptTemplate과 ChatPromptTemplate의 차이점


| 구분 | PromptTemplate      | ChatPromptTemplate    |
|------------|---------------------|---------------|
|  출력 결과   |  문자열(str)  | 메시지 리스트(list[Message])          |
|  입력 구조   |  단일 텍스트   |   role 기반 메시지      |
|  모델 대상   |  LLM / ChatModel   |   ChatModel 전용      |
|  내부 표현   |  텍스트  |   System / Human / AI Message      |

##### - ChatPromptTemplate을 만드는 방법

- 방법 1. 단일 메시지
: `from_template()` 사용 → 내부적으로 역할이 human인 HumanMessagePromptTemplate 사용

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?") 
chat_prompt.format(country="대한민국")

- 방법 2. 여러 역할과 메시지
: `from_messages()` 사용, `system`, `human`, `ai` 메시지 정의

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 방법 2: 여러 역할 및 메시지, from_messages 메서드 사용
chat_template = ChatPromptTemplate.from_messages( 
    [
        # role, message로 tuple 형태로 정의
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name} 입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
) # name, user_input에 값이 없는 경우 error 발생

##### - ChatPromptTemplate 실행방법

- 방법 1. `format_messages()`로 메시지 생성 후 LLM 호출(수동 렌더링)

In [ ]:
# 방법 1. 메시지 생성 후 LLM 호출 
messages = chat_template.format_messages(
    name="테디", user_input="당신의 이름은 무엇입니까?"
)
llm.invoke(messages).content # LLM 호출, AIMessage 객체에서 content로 텍스트 추출(ChatModel이기 때문)


- 방법 2. chain 생성 후 invoke 메서드 사용(자동 렌더링)

In [ ]:
# 방법 2. 체인으로 invoke 메서드 사용하여 LLM 호출
chain = chat_template | llm # chain 생성
chain.invoke({"name": "Teddy", "user_input": "당신의 이름은 무엇입니까?"}).content

#### 3. MessagesPlaceholder

##### - MessagesPlaceholder
 : `ChatPromptTemplate`을 구성요소(컴포넌트) 중 하나로, 여러 메시지(대화 기록)을 삽입하기 위함
 
 → `ChatPromptTemplate` 중간에 `MessagesPlaceholder` 삽입

 → 실행 시점, 대화 기록 개수, 대화 내용 등은 동적으로 삽입됨(실행 시 결정)

##### - MessagesPlaceholder 객체

- `variable_name` : 외부에서 전달받을 변수(명)

- `optional` : 해당 변수가 리스트에 없는 경우 error 발생의 여부
  - `optional = True` : 없는 경우 정상 실행

  - `optional = False` : 없는 경우 error

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다."), # system 메시지
        MessagesPlaceholder(variable_name="conversation"), # MessagesPlaceholder 객체 생성(대화 내용 삽입)
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."), # human 메시지
    ]
)

##### - MessagesPlaceholder 실행방법

- 방법 1. format 메서드로 LLM 호출(수동 렌더링)

In [ ]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5, # 단어 수 지정
    conversation=[ # MessagesPlaceholder에 들어갈 대화 내용 지정
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ], 
)

- 방법 2. chain 생성 후 invoke 메서드 사용(자동 렌더링)

In [ ]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5, # 단어 수 지정
        "conversation": [ # MessagesPlaceholder에 들어갈 대화 내용 지정
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

#### 4. FewShotPromptTemplate

##### - 퓨샷 기법(FewShotPromptTemplate)
: 예제(example)를 프롬프트 앞에 포함하여 모델이 특정 질문-답변을 할 수 있도록 하는 프롬프트 템플릿

 - 제로샷(zero-shot) : 예제 없음

 - 원샷(one-shot) : 하나의 예제

 - 퓨샷(few-shot) : 여러 개의 예제 → FewShotPromptTemplate에서는 퓨샷을 대부분 사용함

##### - FewShotPromptTemplate 객체
 - `examples` : 예제 데이터 목록
 
 - `example_prompt` : 예제 데이터를 어떻게 LLM이 읽을 수 있는 형태로 변환할 것인지 정하는 규칙

 - `suffix` : 실제 LLM에게 할 질문

 - `input_variables` : 입력받아야 할 변수 목록

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

examples = [ # 예제 질문(key)-답변(value)(list of dict 형태)
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

- 예제 렌더링 방법

In [ ]:
example_prompt = PromptTemplate.from_template( # PromptTemplate 객체 생성
    "Question:\n{question}\nAnswer:\n{answer}"
)

example_prompt.format(**examples[0]) # 프롬프트 생성
# 딕셔너리 언패킹(**)를 사용하여 examples[0]의 question, answer 값을 각각 치환
# example_prompt.format(
#     question=examples[0]["question"],
#     answer=examples[0]["answer"]
# ) 과 동일

##### - FewShotPromptTemplate 실행 방법

 - 방법 1. format()으로 프롬프트 문자열 생성(수동 렌더링)


In [ ]:
prompt = FewShotPromptTemplate( # FewShotPromptTemplate 객체 생성
    examples=examples, 
    example_prompt=example_prompt, # 각 예제에 대해 내부적으로 format(**example)을 실행
    suffix="Question:\n{question}\nAnswer:", #실제 질문-답변 템플릿
    input_variables=["question"], # 입력받을 변수 지정
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question) # format 메서드로 프롬프트 생성

from langchain_openai import ChatOpenAI
from langchain_teddynote.prompts import stream_response

llm = ChatOpenAI()

# 결과 출력
answer = llm.stream(final_prompt)
stream_response(answer)

 - 방법 2. chain 생성 후 invoke로 실행(자동 렌더링)

In [ ]:
# chain 생성
chain = prompt | llm | StrOutputParser() # 프롬프트 렌더링 + LLM 호출 + 출력 파싱

# 결과 출력
answer = chain.stream(
    {"question": "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"}
)
stream_response(answer)

#### 5. 예제 선택기

##### - SemanticSimilarityExampleSelector
: 질문과 유사한 예시를 임베딩 기반 유사도 검색 후 example 목록에서 찾아 상위 k개를 선택

→ 다양성이 부족하여 비슷한 예제로 중복될 수 있음

→ 사용자 지정 임베딩 방법과 코사인 유사도 기반 계산

##### - SemanticSimilarityExampleSelector 객체

 - `examples` : 예제 데이터 목록

 - `embedding` : 텍스트를 벡터로 변환하는 임베딩 모델

 - `vectorstore_cls` : 임베딩 벡터 저장 및 유사도 검색하는 VectorStore(저장소, DB)

 - `k` : 선택할 예제 개수(상위 k개)

In [ ]:
from langchain_core.example_selectors import (MaxMarginalRelevanceExampleSelector,SemanticSimilarityExampleSelector)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("example_selector", OpenAIEmbeddings())

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(), # OpenAI text embedding 모델 사용
    Chroma, # Vector DB
    k=1, # 상위 1개
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

# 질문과 예제간의 SemanticSimilarityExampleSelector를 이용한 유사도 계산
selected_examples = example_selector.select_examples({"question": question})

# 어떤 예시가 선택되었는지 출력
print(f"입력에 가장 유사한 예시:\n{question}\n")
for example in selected_examples:
    print(f'question:\n{example["question"]}')
    print(f'answer:\n{example["answer"]}')

`ExampleSelector` 를 사용하여 `FewShotPromptTemplate` 생성 → 프롬프트에 포함하여 최종 질문-답변 형식의 템플릿

In [ ]:
# 선택된 예시로 FewShotPromptTemplate 객체 생성
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
example_selector_prompt = prompt.format(question=question)

##### - MaxMarginalRelevanceExampleSelector
: 유사도과 예시의 다양성까지 고려하여 example 목록에서 찾아 선택

→ 사용자에게 다양하고 관련성있는 정보를 제공하여 커버리지가 좋아짐

1. 관련성(relavance) : 사용자가 입력한 검색어, 주제, 문서와 잘 맞는지를 평가하는 기준

2. 다양성(diversity) : 선택된 예제와 새로운 예제 간의 유사성 계산

MMR은 질문과의 유사도와 이미 선택된 예제들과의 중복을 동시에 고려하여, 다음 식과 같이 예제를 선택한다.

$$
\text{MMR}
=
\arg\max_{d_i \in R \setminus S}
\left[
\lambda \cdot \text{Sim}(d_i, Q)
-
(1 - \lambda) \cdot \max_{d_j \in S} \text{Sim}(d_i, d_j)
\right]
$$

- $R$ : 전체 후보 예제 집합 

- $S$ : 이미 선택된 예제 집합  

- $Q$ : 입력 질문  

- $\text{Sim}(\cdot)$ : 코사인 유사도  

- $\lambda$ : 유사도와 다양성 간의 균형 계수, 값이 클수록 관련성 중시, 값이 작을수록 다양성 중시

#### 6. FewShotChatMessagePromptTemplate

##### - FewShotChatMessagePromptTemplate
 : FewShotPromptTemplate과 ChatMessage를 결합하여 대화형 프롬프트에서 few-shot을 메시지 단위로 구성하기 위한 메시지 프롬프트 템플릿

##### - FewShotChatMessagePromptTemplate 객체
 - `examples` : 예제 데이터 목록

 - `example_prompt` : 예제 데이터를 어떻게 LLM이 읽을 수 있는 형태로 변환할 것인지 정하는 규칙

In [ ]:
examples = [ # 예제 질문(key)-답변(value)(list of dict 형태)
    {
        "instruction": "당신은 회의록 작성 전문가 입니다. 주어진 정보를 바탕으로 회의록을 작성해 주세요",
        "input": "2023년 12월 25일, XYZ 회사의 마케팅 전략 회의가 오후 3시에 시작되었다. 회의에는 마케팅 팀장인 김수진, 디지털 마케팅 담당자인 박지민, 소셜 미디어 관리자인 이준호가 참석했다. 회의의 주요 목적은 2024년 상반기 마케팅 전략을 수립하고, 새로운 소셜 미디어 캠페인에 대한 아이디어를 논의하는 것이었다. 팀장인 김수진은 최근 시장 동향에 대한 간략한 개요를 제공했으며, 이어서 각 팀원이 자신의 분야에서의 전략적 아이디어를 발표했다.",
        "answer": """
회의록: XYZ 회사 마케팅 전략 회의
일시: 2023년 12월 25일
장소: XYZ 회사 회의실
참석자: 김수진 (마케팅 팀장), 박지민 (디지털 마케팅 담당자), 이준호 (소셜 미디어 관리자)

1. 개회
   - 회의는 김수진 팀장의 개회사로 시작됨.
   - 회의의 목적은 2024년 상반기 마케팅 전략 수립 및 새로운 소셜 미디어 캠페인 아이디어 논의.

2. 시장 동향 개요 (김수진)
   - 김수진 팀장은 최근 시장 동향에 대한 분석을 제시.
   - 소비자 행동 변화와 경쟁사 전략에 대한 통찰 공유.

3. 디지털 마케팅 전략 (박지민)
   - 박지민은 디지털 마케팅 전략에 대해 발표.
   - 온라인 광고와 SEO 최적화 방안에 중점을 둠.

4. 소셜 미디어 캠페인 (이준호)
   - 이준호는 새로운 소셜 미디어 캠페인에 대한 아이디어를 제안.
   - 인플루언서 마케팅과 콘텐츠 전략에 대한 계획을 설명함.

5. 종합 논의
   - 팀원들 간의 아이디어 공유 및 토론.
   - 각 전략에 대한 예산 및 자원 배분에 대해 논의.

6. 마무리
   - 다음 회의 날짜 및 시간 확정.
   - 회의록 정리 및 배포는 박지민 담당.
""",
    },
    {
        "instruction": "당신은 요약 전문가 입니다. 다음 주어진 정보를 바탕으로 내용을 요약해 주세요",
        "input": "이 문서는 '지속 가능한 도시 개발을 위한 전략'에 대한 20페이지 분량의 보고서입니다. 보고서는 지속 가능한 도시 개발의 중요성, 현재 도시화의 문제점, 그리고 도시 개발을 지속 가능하게 만들기 위한 다양한 전략을 포괄적으로 다루고 있습니다. 이 보고서는 또한 성공적인 지속 가능한 도시 개발 사례를 여러 국가에서 소개하고, 이러한 사례들을 통해 얻은 교훈을 요약하고 있습니다.",
        "answer": """
문서 요약: 지속 가능한 도시 개발을 위한 전략 보고서

- 중요성: 지속 가능한 도시 개발이 필수적인 이유와 그에 따른 사회적, 경제적, 환경적 이익을 강조.
- 현 문제점: 현재의 도시화 과정에서 발생하는 주요 문제점들, 예를 들어 환경 오염, 자원 고갈, 불평등 증가 등을 분석.
- 전략: 지속 가능한 도시 개발을 달성하기 위한 다양한 전략 제시. 이에는 친환경 건축, 대중교통 개선, 에너지 효율성 증대, 지역사회 참여 강화 등이 포함됨.
- 사례 연구: 전 세계 여러 도시의 성공적인 지속 가능한 개발 사례를 소개. 예를 들어, 덴마크의 코펜하겐, 일본의 요코하마 등의 사례를 통해 실현 가능한 전략들을 설명.
- 교훈: 이러한 사례들에서 얻은 주요 교훈을 요약. 강조된 교훈에는 다각적 접근의 중요성, 지역사회와의 협력, 장기적 계획의 필요성 등이 포함됨.

이 보고서는 지속 가능한 도시 개발이 어떻게 현실적이고 효과적인 형태로 이루어질 수 있는지에 대한 심도 있는 분석을 제공합니다.
""",
    },
    {
        "instruction": "당신은 문장 교정 전문가 입니다. 다음 주어진 문장을 교정해 주세요",
        "input": "우리 회사는 새로운 마케팅 전략을 도입하려고 한다. 이를 통해 고객과의 소통이 더 효과적이 될 것이다.",
        "answer": "본 회사는 새로운 마케팅 전략을 도입함으로써, 고객과의 소통을 보다 효과적으로 개선할 수 있을 것으로 기대된다.",
    },
]

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.example_selectors import (
    SemanticSimilarityExampleSelector,
)
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

chroma = Chroma("fewshot_chat", OpenAIEmbeddings()) # VectorDB 생성

example_prompt = ChatPromptTemplate.from_messages( # 프롬프트 정의
    [ 
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
) # [HumanMessage, AIMessage] 리스트 형태로 구성

example_selector = SemanticSimilarityExampleSelector.from_examples( 
    examples, # 예시 목록
    OpenAIEmbeddings(), # 임베딩
    chroma, # VectorDB
    k=1, # 상위 1개
)

few_shot_prompt = FewShotChatMessagePromptTemplate( 
    example_selector=example_selector, # 예시 선택기(가장 유사한 예시 1개 선택)
    example_prompt=example_prompt, # 예시 프롬프트
)

##### - FewShotPromptTemplate과 FewShotChatMessagePromptTemplate의 차이

- FewShotPromptTemplate : 문자열 기반 프롬프트 

- FewShotChatMessagePromptTemplate : Chat모델에서 메시지(role)단위를 유지

서로 분리된 구조를 지님

#### 7. CustomExampleSelector

##### - CustomExampleSelector
: 사용자가 직접 정의한 규칙으로 입력과 적절한 예제를 고르는 예제 선택기


##### - CustomExampleSelector 객체

 - `examples` : 예제 데이터 목록

 - `embedding` : 텍스트를 벡터로 변환하는 임베딩 모델

In [ ]:
from langchain_teddynote.prompts import CustomExampleSelector

# 커스텀 예제 선택기 생성
custom_selector = CustomExampleSelector(examples, OpenAIEmbeddings())

# 커스텀 예제 선택기를 사용했을 때 결과
custom_selector.select_examples({"instruction": "다음 문장을 회의록 작성해 주세요"})

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{instruction}:\n{input}"),
        ("ai", "{answer}"),
    ]
)

custom_fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=custom_selector,  # 커스텀 예제 선택기 사용
    example_prompt=example_prompt,  # 예제 프롬프트 사용
)

custom_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        custom_fewshot_prompt,
        ("human", "{instruction}\n{input}"),
    ]
)

# chain 을 생성합니다.
chain = custom_prompt | llm

question = {
    "instruction": "회의록을 작성해 주세요",
    "input": "2023년 12월 26일, ABC 기술 회사의 제품 개발 팀은 새로운 모바일 애플리케이션 프로젝트에 대한 주간 진행 상황 회의를 가졌다. 이 회의에는 프로젝트 매니저인 최현수, 주요 개발자인 황지연, UI/UX 디자이너인 김태영이 참석했다. 회의의 주요 목적은 프로젝트의 현재 진행 상황을 검토하고, 다가오는 마일스톤에 대한 계획을 수립하는 것이었다. 각 팀원은 자신의 작업 영역에 대한 업데이트를 제공했고, 팀은 다음 주까지의 목표를 설정했다.",
}
stream_response(chain.stream(question))

#### 8. 출력 파서(Output Parser)

##### - 출력 파서(Output Parser)
: LLM 모델의 출력값을 구조화된 형식으로 변환하고 원하는 정보를 추출할 수 있는 요소 
→ JSON 형식으로 구조화된 답변을 받을 수 있음(key-value 형태)

##### - 출력 파서의 특징
 1. 다양성 : 다양한 종류의 출력 파서 제공

 2. 스트리밍 지원 : 실시간 데이터 처리 가능

 3. 확장성 : 확장 가능한 인터페이스 제공

##### - 출력 파서의 장점

 1. 구조화 : 출력을 구조화된 데이터로 변환하여 체계적인 정보 관리

 2. 일관성 : 일관된 출력 형식

 3. 유연성 : JSON, 딕셔너리, 리스트 등 다양한 출력 형식 변환 가능

#### 9. PydanticOutPutParser

##### - Pydantic 모델
 : 데이터의 구조(필드), 타입, 제약조건을 정의한 설계도(schema)

##### - Pydantic 객체
 : Pydantic 모델을 기준으로 실제 값이 채워진 실제 데이터

##### - PydanticOutPutParser 
: LLM의 텍스트 출력을 특정 Pydantic 데이터 모델(schema)에 맞게 구조화된 객체로 변환

 - Pydantic : 데이터 구조 정의 타입 검증 및 변환 자동 수행하는 라이브러리
 → 원하는 정보만 정확한 구조, 타입으로 출력하기 위함 

 - 스키마(schema) : 데이터가 가져야 할 구조, 규칙을 정의

##### - PydanticOutPutParser 주요 메서드
 - `get_format_instructions()` : LLM이 출력해야 할 정보 형식 정의 ex) 데이터 필드
 → Pydantic 모델에 맞는 출력을 유도

 - `parse()` : LLM 출력에서 정해진 규칙(schema)을 기준으로 의미 있는 값을 뽑아 Pydantic 모델에 넣고, 스키마 검증 후 객체로 변환

##### - PydanticOutPutParser 객체
 - `pydantic_object` : LLM 출력이 따라야 할 데이터 모델(schema)

In [ ]:
# Pydantic을 사용한 출력 파싱 예제

from pydantic import BaseModel

class Person(BaseModel): # 이때 Person 클래스는 Pydantic 모델
    name: str
    age: int

person = Person(name="테디", age=20) # Person 객체는 Pydantic 객체

In [ ]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field # Pydantic 모델 정의에 사용

llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

In [ ]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me) 
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [ ]:
class EmailSummary(BaseModel): # Pydantic 모델 정의
    # 각 description은 필드에 대한 설명을 제공(LLM이 해당 설명을 보고 정보 추출)
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

parser.get_format_instructions()

프롬프트 정의

1. `question`: 유저의 질문을 받음

2. `email_conversation`: 이메일 본문의 내용을 입력

3. `format`: 형식을 지정


In [ ]:
# prompt 템플릿 생성
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

##### - LLM 출력 파싱 방법

- 방법 1. LLM 출력 결과로 (수동 파싱)

In [ ]:
# format에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

# chain 생성
chain = prompt | llm

# chain 실행 및 결과 출력
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# JSON 형태로 출력
output = stream_response(response, return_output=True) # True면 스트리밍 + 문자열 False면 스트리밍만 반환

# PydanticOutputParser를 사용하여 결과 파싱
structured_output = parser.parse(output) # LLM이 출력한 문자열 output을 Pydantic 객체로 파싱

- 방법 2. Chain에 Parser를 구성하여 (자동 파싱)

In [ ]:
# 출력 파서를 추가하여 전체 체인 재구성
chain = prompt | llm | parser

# chain 실행 및 결과 출력
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# EmailSummary 객체 형태로 출력
response

#### 10. with_structured_output() 바인딩

: `with_structured_output(Pydantic)`을 사용하여 출력을 Pydantic 객체 변환

In [ ]:
llm_with_structered = ChatOpenAI(
    temperature=0, model_name="gpt-4.1-mini"
).with_structured_output(EmailSummary) 

# invoke() 함수를 호출하여 결과를 출력합니다.
answer = llm_with_structered.invoke(email_conversation) # 출력 결과를 바로 Pydantic 객체로 반환
answer

##### - PydanticOutputParser과 with_structured_output()의 차이
 - `PydanticOutputParser` : LLM 응답 생성 후 출력한 결과를 가지고 Pydantic 객체로 변환

 - `with_structured_output()` : LLM 응답 생성 과정에서 Pydantic 객체로 변환


| 구분 | PydanticOutputParser       | with_structured_output()    |
|------------|---------------------|---------------|
|  적용 시점   | 생성 후 단계(후처리)        | 생성 단계(전처리)          |
|  어디에 붙나   | 모델    | 체인        |

#### 11. 쉼표 구분된 리스트 출력 파서

##### - 쉼표 구분된 리스트 출력 파서(CommaSeparatedListOutputParser)
 : 쉼표(,)로 구분된 LLM의 출력 결과인 문자열을 리스트로 변환하는 출력 파서

 ex) 서울, 부산, 경기도 → ["서울", "부산", "경기도"]


##### - CommaSeparatedListOutputParser 객체
- `get_format_instructions()` : LLM이 출력해야 할 정보 형식 정의 ex) 데이터 필드
 → Pydantic 모델에 맞는 출력을 유도

 - `parse()` : LLM 출력에서 정해진 규칙(schema)을 기준으로 의미 있는 값을 뽑아 Pydantic 모델에 넣고, 스키마 검증 후 객체로 변환

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 변환
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

model = ChatOpenAI(temperature=0)

# chain 생성
chain = prompt | model | output_parser

# list 반환
chain.invoke({"subject": "대한민국 관광명소"})

for s in chain.stream({"subject": "대한민국 관광명소"}):
    print(s)  # 스트림의 내용을 출력, 활용성 떨어짐

#### 12. 구조화된 출력 파서

##### - 구조화된 출력 파서(StructuredOutputParser)
 : JSON 형식이 과도하거나 불안정한 상황에서 최소한의 key-value 구조를 출력하는 출력 파서
  
  → GPT, Claude 모델 보다 인텔리전스가 낮은 로컬 모델에 사용, JSON/Pydantic보다 가벼워 간단한 경우 사용함

##### - StructuredOutputParser 객체
: LLM의 출력을 사전에 정의한 필드(schema)에 맞춰 딕셔너리 형태로 변환하는 객체

- `response_schemas` : LLM 출력에 포함되어야 할 필드 목록

##### - ResponseSchema 객체
: LLM 응답에 포함되어야 할 필드의 이름, 의미를 정의

- `name` : 출력 결과에서 사용할 key 이름

- `description` : LLM에게 해당 필드에 대한 설명

In [ ]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# LLM 출력에 반드시 포함되어야 할 필드 목록 정의
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]

# 응답을 구조화한 출력 파서 생성
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 출력 형식 지시사항 파싱
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions},
) # 프롬프트 템플릿 생성

model = ChatOpenAI(temperature=0)
chain = prompt | model | output_parser # chain 생성

# dict 반환
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})

for s in chain.stream({"question": "세종대왕의 업적은 무엇인가요?"}):
    # 스트리밍 출력
    print(s)

#### 13. JSON 형식 출력 파서

##### - JSON 형식 출력 파서(JsonOutputParser)
: LLM의 출력을 JSON 형식의 데이터를 추출하여 구조화된 Python 딕셔너리(dict)로 변환하는 출력 파서
→ 용량이 작은 모델에서 JsonOutputParser를 사용할 경우 오류가 발생할 수 있음

##### - JsonOutputParser 객체

 - `get_format_instructions()` : LLM이 출력해야 할 정보 형식 정의 ex) 데이터 필드
  → Pydantic 모델에 맞는 출력을 유도

 - `parse()` : LLM의 출력 텍스트를 Pydantic 모델 기준으로 파싱·검증하여 객체로 변환

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

model = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

# Pydantic 모델 정의
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

question = "지구 온난화의 심각성 대해 알려주세요."

parser = JsonOutputParser(pydantic_object=Topic) # JsonOutputParser 생성
print(parser.get_format_instructions()) 

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)
prompt = prompt.partial(format_instructions=parser.get_format_instructions()) # ChatPromptTemplate를 수정한 새로운 프롬프트 생성

# 체인 구성
chain = prompt | model | parser
answer = chain.invoke({"question": question})

type(answer)
answer # dict 형태

##### - StructuredOutputParser과 JsonOutputParser 차이

- `StructuredOutputParser`
: 출력 형식 요구가 단순하여 출력 제어가 약한 로컬·소형 모델에서도 안정적으로 파싱 가능

- `JsonOutputParser`
: JSON 문법을 정확히 지켜야 하며 형식이 조금이라도 깨질 경우 파싱 오류 발생

#### 14. Pandas 데이터프레임 출력 파서

##### - Pandas 데이터프레임 출력 파서(PandasDataFrameOutputParser)
 : LLM 출력 결과를 Pandas Dataframe 객체로 변환하는 출력 파서

##### - PandasDataFrameOutputParser 객체
 
 - `dataframe_columns` : DataFrame에 포함되어야 할 열(column) 목록, None일 경우 임시 컬럼명 및 첫 줄을 열로 판단

In [ ]:
import pprint
from typing import Any, Dict
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# 출력 목적으로 사용
def format_parser_output(parser_output: Dict[str, Any]) -> None:
    # 파서 출력 키 순회
    for key in parser_output.keys():
        # 각 키의 값을 딕셔너리로 변환
        parser_output[key] = parser_output[key].to_dict()
    return pprint.PrettyPrinter(width=4, compact=True).pprint(parser_output) # 들여쓰기 4칸, 한 줄로 출력

df = pd.read_csv("./data/titanic.csv") # 데이터 불러오기
df.head()

parser = PandasDataFrameOutputParser(dataframe=df)
print(parser.get_format_instructions())

df_query = "Age column 을 조회해 주세요." # Age 열 조회

# 프롬프트 템플릿 생성
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{question}\n",
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

chain = prompt | model | parser # chain 생성

parser_output = chain.invoke({"question": df_query})
format_parser_output(parser_output)

# 행 조회
df_query = "Retrieve the first row."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

# row 0 ~ 4의 평균 나이 계산
df["Age"].head().mean()

# Pandas DataFrame 작업 예시, 행의 수를 제한
df_query = "Retrieve the average of the Ages from row 0 to 4."

# 체인 실행
parser_output = chain.invoke({"question": df_query})

#### 15. 날짜 형식 출력 파서

##### - 날짜 형식 출력 파서(DatetimeOutputParser)
 : LLM의 출력을 datetime 형식으로 변환하는 출력 파서

##### - DatetimeOutputParser 객체
 - `format` : LLM 출력이 따라야 할 날짜/시간 문자열 포맷, None일 경우 자동적으로 날짜 포맷 시도

##### - datetime 객체
 : 날자와 시간을 하나의 구조로 표현하는 자료형, (year, month, day)로 구성

**참고**

| 형식 코드 | 설명                | 예시          |
|------------|---------------------|---------------|
| %Y         | 4자리 연도          | 2024          |
| %y         | 2자리 연도          | 24            |
| %m         | 2자리 월            | 07            |
| %d         | 2자리 일            | 04            |
| %H         | 24시간제 시간       | 14            |
| %I         | 12시간제 시간       | 02            |
| %p         | AM 또는 PM          | PM            |
| %M         | 2자리 분            | 45            |
| %S         | 2자리 초            | 08            |
| %f         | 마이크로초 (6자리)  | 000123        |
| %z         | UTC 오프셋          | +0900         |
| %Z         | 시간대 이름         | KST           |
| %a         | 요일 약어           | Thu           |
| %A         | 요일 전체           | Thursday      |
| %b         | 월 약어             | Jul           |
| %B         | 월 전체             | July          |
| %c         | 전체 날짜와 시간     | Thu Jul  4 14:45:08 2024 |
| %x         | 전체 날짜           | 07/04/24      |
| %X         | 전체 시간           | 14:45:08      |

In [ ]:
from langchain.output_parsers import DatetimeOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 날짜 및 시간 출력 파서
output_parser = DatetimeOutputParser() # datetime 객체 반환
output_parser.format = "%Y-%m-%d"

# 사용자 질문에 대한 답변 템플릿
template = """Answer the users question:\n\n#Format Instructions: \n{format_instructions}\n\n#Question: \n{question}\n\n#Answer:"""

prompt = PromptTemplate.from_template(
    template,
    partial_variables={
        "format_instructions": output_parser.get_format_instructions()
    },  # 지침을 템플릿에 적용
)

# 프롬프트 내용 출력
prompt

# chain 생성
chain = prompt | ChatOpenAI() | output_parser
output = chain.invoke({"question": "Google 이 창업한 연도"})

# 문자열 변환
output.strftime("%Y-%m-%d")

#### 16. 열거형 출력 파서

##### - 열거형 출력 파서(EnumOutputParser)
 : LLM 출력을 미리 정의된 Enum 값을 기준으로 문자열을 출력하는 출력 파서 → 출력 데이터의 일관성 유지
 - Enum : 동등한 레벨에 있는 데이터들을 하나의 구조로 묶어 표현

##### - EnumOutputParser 객체
 - `enum` : LLM 출력이 반드시 일치해야 할 값들의 집합

In [ ]:
from langchain.output_parsers.enum import EnumOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from enum import Enum

# 선택가능한 값들을 정의하는 Enum 클래스 생성(LLM의 출력은 반드시 이 값들 중 하나여야 함)
class Colors(Enum):
    RED = "빨간색"
    GREEN = "초록색"
    BLUE = "파란색"

# EnumOutputParser 생성
parser = EnumOutputParser(enum=Colors) # Colors Enum 클래스 값 제외 모두 허용하지 않음, 있는 경우 error 발생

# 프롬프트 템플릿 생성
prompt = PromptTemplate.from_template(
    """다음의 물체는 어떤 색깔인가요?

Object: {object}

Instructions: {instructions}"""
).partial(instructions=parser.get_format_instructions())

# 체인 생성
chain = prompt | ChatOpenAI() | parser

# 체인 실행
response = chain.invoke({"object": "하늘"})
type(response) # Enum 타입 <class 'enum.Enum'>
response.value # Enum 값 '파란색'('하늘'에 대한 답변)